# Dataset Creation

In [1]:
#Clear previously stored variables
%reset -f

## 1. Imports

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib

## 2. Creating Lists of Image Paths and MGMT Labels


### 2.1 Glob Function

In [3]:
import glob # for loading the data
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

### 2.2 Dataset - Ivy Foundation


#### 2.2.1 Loading Labels from Excel


In [39]:
ivy_data = pd.read_excel('/Volumes/TRIALS/Ivy_Foundation/Document_final/Ivy_OS_Data_September2018_JEG_edits.xlsx')
ivy_data = ivy_data[['ID', 'MGMT']]

ivy_labels = {}

for i in ivy_data.index:
    if (ivy_data['MGMT'][i] == "Methylated"):
        ivy_labels[ivy_data['ID'][i]] = True
    elif (ivy_data['MGMT'][i] == "Unmethylated"):
        ivy_labels[ivy_data['ID'][i]] = False


#### 2.2.2 Adding Labels and Image Path to List Together

In [43]:
ivy_root = '/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution'
#ivy_flair_path = ivy_root + '/*/*/NIFTI_Native/flair.nii.gz'
#ivy_flair_list = insensitive_glob(ivy_flair_path)
ivy_list = []

for key in ivy_labels.keys():
    ivy_list.append((ivy_root + '/Post/' + "{:05d}".format(key) + '/NIFTI_Native/flair.nii.gz', ivy_labels[key]))

print(ivy_list)

[('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00006/NIFTI_Native/flair.nii.gz', False), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00010/NIFTI_Native/flair.nii.gz', True), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00017/NIFTI_Native/flair.nii.gz', True), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00025/NIFTI_Native/flair.nii.gz', False), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00026/NIFTI_Native/flair.nii.gz', True), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00029/NIFTI_Native/flair.nii.gz', False), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00031/NIFTI_Native/flair.nii.gz', False), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00034/NIFTI_Native/flair.nii.gz', False), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00054/NIFTI_Native/flair.nii.gz', False), ('/Volumes/TRIALS/Ivy_Foundation/Final_for_Distribution/Post/00057/NIFTI_Na